In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import Compose, Resize, ToTensor, ToPILImage

from torchvision.models.inception import Inception3
from torch.utils.model_zoo import load_url as load_state_dict_from_url

import numpy as np
import pandas as pd

import os

import cv2
import imageio
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
class positionalEncoder(nn.Module):

  def __init__(self, frame_length, encoding_length):
    super().__init__()

    self.embedding = nn.Embedding(frame_length, encoding_length)
    
    self.frame_length = frame_length

  def forward(self, x):

    pe = self.embedding(torch.tensor([i for i in range(self.frame_length)]))

    pe = pe[:x.shape[0]]

    if len(x.shape) == 3:
      self.pe = pe.unsqueeze(0).repeat(x.shape[0], 1, 1)
      x = torch.cat((x, pe), 2)
    else:
      x = torch.cat((x, pe), 1)


    return x

In [3]:
inc_v3_url = 'https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth'

def inception_v3_sliced(stop_layer=3, **kwargs):

    kwargs['transform_input'] = True
    kwargs['init_weights'] = False

    class Inception3Mod(Inception3):

        def __init__(self, stop_layer, **kwargs):
            super(Inception3Mod, self).__init__(**kwargs)
            self.stop_layer = stop_layer
            
        def _forward(self, x):
            layers = [
            self.Conv2d_1a_3x3, self.Conv2d_2a_3x3, self.Conv2d_2b_3x3,
            'maxpool',
            self.Conv2d_3b_1x1, self.Conv2d_4a_3x3,
            'maxpool',
            self.Mixed_5b, self.Mixed_5c, self.Mixed_5d, self.Mixed_6a,
            self.Mixed_6b, self.Mixed_6c, self.Mixed_6d, self.Mixed_6e,
            self.Mixed_7a, self.Mixed_7b, self.Mixed_7c]

            for idx in range(self.stop_layer):
                layer = layers[idx]
                if layer == 'maxpool':
                    x = nn.functional.max_pool2d(x, kernel_size=3, stride=2)
                else:
                    x = layer(x)
            return x, None
        
    model = Inception3Mod(**kwargs, stop_layer=stop_layer)
    state_dict = load_state_dict_from_url(inc_v3_url, progress=True)
    model.load_state_dict(state_dict)
    model.aux_logits = False
    del model.AuxLogits
    return model

In [4]:
class encoderTransformer(nn.Module):

  def __init__(self, num_T_layers, num_fc_layers, outFeatCount, device, testLayer = 1, num_frames_max = 1000, stopLayer = 16, train = True, pos_encode_size = 10, n_hidden = 2048, n_heads = 86, dropout = 0.3):
    super().__init__()

    self.posEncoder = positionalEncoder(num_frames_max, pos_encode_size)
    
    self.visionEncoder = inception_v3_sliced(stop_layer=stopLayer)
    
    self.transform = Compose([ToPILImage(), Resize((299, 299)), ToTensor()])

    inFeatCount = 1280 + pos_encode_size
    
    encoder_layer = nn.TransformerEncoderLayer(inFeatCount, n_heads, n_hidden, dropout)
    self.encoder = nn.TransformerEncoder(encoder_layer, num_T_layers)

    self.fcLayers = []
    currInput = inFeatCount

    for i in range(num_fc_layers):
      if i == num_fc_layers - 1:
        self.fcLayers.append(nn.Linear(currInput, outFeatCount))
      else:
        self.fcLayers.append(nn.Linear(currInput, currInput // 2))
        currInput = currInput // 2

    print(f"{len(self.fcLayers)} fc layers")

    self.device = device
    self.train = train
    
    self.outputfc = min(testLayer, len(self.fcLayers))

    print(f'output layer count: {self.outputfc}')
    
    self.init_weights()

  def init_weights(self):
      initrange = 0.1

      for i in self.fcLayers:
        i.bias.data.zero_()
        i.weight.data.uniform_(-initrange, initrange)

  def forward(self, data, vid = False, toCSV = False, toCSVName = None, toCSVPath = None):

    arr = None
    
    if vid:
      imgArr = []
      vid = imageio.get_reader(data)
      for i, im in enumerate(vid):
        imgArr.append(im)
    
    elif not vid:
      imgArr = data

    for i in imgArr:
      img = self.transform(i)[None]
      picToVal = self.visionEncoder(img)[0]
      to1D = nn.MaxPool2d((8, 8))(picToVal).squeeze()
      if arr == None:
        arr = to1D[None, :]
      else:
        arr = torch.cat((arr,to1D[None, :]), 0)

    if vid: os.system("rm ./*.jpg")
  
    encoded = self.posEncoder(arr) #make sure this works

    result = self.encoder(encoded)

    if self.train:
      for i in self.fcLayers:
        result = i(result)
    else:
      for i in range(min(self.outputfc, len(self.fcLayers))):
        result = self.fcLayers[i](result)
        if toCSV:
          pd.DataFrame(result.numpy()).to_csv(f'{toCSVPath}{toCSVName}.csv')
          return
          
    return result

In [5]:
openface = pd.read_csv("data/OpenFace/trial/lie/trial_lie_001.csv")
openface = openface[['gaze_0_x','gaze_0_y','gaze_0_z','gaze_angle_x', 'gaze_angle_y','AU01_r','AU02_r','AU04_r','AU05_r'\
,'AU06_r','AU07_r','AU09_r','AU10_r','AU12_r','AU14_r']]
openface = torch.tensor(np.array(openface)).float()

encoder_loss_fuction = nn.MSELoss()

In [6]:
def preprocess(path):
    imgArr = []
    Openface_Arr = pd.DataFrame()
    idx = 0
    for video in os.listdir(path):
        if video == 'trial_lie_043.mp4':
            continue
        if idx == 10:
            break
        if video.endswith(".mp4"):
            print('Processing video: ', video, 'finished: ', idx)
            vid = imageio.get_reader(path + "/" + video)
            for frame_number, im in enumerate(vid):
                im = cv2.resize(im, (240, 320))
                imgArr.append(im)
            
            if video[:-4] + ".csv" in os.listdir("data/OpenFace/trial/lie/"):
                file = pd.read_csv("data/OpenFace/trial/lie/" + video[:-4] + ".csv")
                file = file[['gaze_0_x','gaze_0_y','gaze_0_z','gaze_angle_x',
                                'gaze_angle_y','AU01_r','AU02_r','AU04_r','AU05_r',
                                'AU06_r','AU07_r','AU09_r','AU10_r','AU12_r','AU14_r']]
                Openface_Arr = pd.concat([Openface_Arr, file])
            if video[:-4] + ".csv" in os.listdir("data/OpenFace/trial/truth/"):
                file = pd.read_csv("data/OpenFace/trial/truth/" + video[:-4] + ".csv")
                file = file[['gaze_0_x','gaze_0_y','gaze_0_z','gaze_angle_x',
                                'gaze_angle_y','AU01_r','AU02_r','AU04_r','AU05_r',
                                'AU06_r','AU07_r','AU09_r','AU10_r','AU12_r','AU14_r']]
                Openface_Arr = pd.concat([Openface_Arr, file])

            idx += 1

    return imgArr, np.array(Openface_Arr)

imgArr, Openface_Arr = preprocess("../Videos/")

Processing video:  trial_lie_041.mp4 finished:  0
Processing video:  trial_lie_055.mp4 finished:  1
Processing video:  trial_truth_056.mp4 finished:  2
Processing video:  trial_truth_042.mp4 finished:  3
Processing video:  trial_truth_043.mp4 finished:  4
Processing video:  trial_truth_057.mp4 finished:  5
Processing video:  trial_lie_054.mp4 finished:  6
Processing video:  trial_lie_040.mp4 finished:  7
Processing video:  trial_lie_056.mp4 finished:  8
Processing video:  trial_lie_042.mp4 finished:  9


In [7]:
def train_test_split(imgArr, Openface_Arr, test_size, random_state):
    np.random.seed(random_state)
    idx = np.random.permutation(len(imgArr))
    test_size = int(len(imgArr) * test_size)
    X_train = [imgArr[i] for i in idx[test_size:]]
    X_test = [imgArr[i] for i in idx[:test_size]]
    y_train = [Openface_Arr[i] for i in idx[test_size:]]
    y_test = [Openface_Arr[i] for i in idx[:test_size]]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(imgArr, Openface_Arr, test_size=0.2, random_state=42)

In [8]:
model = encoderTransformer(1, 1, 15, 'cpu')

print(np.array(X_train[0:10]).shape)
model(X_train[0:10])

1 fc layers
output layer count: 1
(10, 320, 240, 3)


tensor([[-7.5498e-02, -2.8750e+00,  1.8817e+00,  1.0084e+00,  6.1173e-01,
         -2.6245e+00,  3.3684e-01, -4.7728e-01,  2.2174e+00,  9.6956e-01,
         -1.3995e+00,  1.1406e+00,  5.3648e-01, -1.1025e+00,  2.4141e+00],
        [-1.9975e+00,  3.0389e+00, -1.4600e+00, -1.0495e+00,  1.7765e+00,
         -1.2133e+00, -1.7972e+00,  3.0130e+00, -2.6052e-01,  2.8389e+00,
          1.2919e+00,  3.6225e+00, -2.0970e+00, -3.9113e-03,  2.6600e-01],
        [ 3.8864e-01, -9.5516e-01,  1.1841e+00,  1.7247e-01,  2.7511e-01,
         -4.2051e-01, -9.5092e-01,  2.4355e+00,  1.2049e+00,  3.6847e+00,
          3.3409e-01,  9.1909e-01, -2.0130e+00, -2.7060e+00,  3.7502e+00],
        [ 1.4020e+00, -2.8604e-01,  5.1584e-01, -2.3447e+00,  2.2565e-02,
          3.8842e-01,  4.7095e-02,  3.3504e+00,  1.0034e+00,  2.0034e+00,
          1.2500e+00,  2.2443e+00,  4.8683e-01, -3.2714e+00,  1.1672e+00],
        [-1.0307e+00, -1.5193e+00, -2.1108e+00, -2.0884e+00,  8.2929e-01,
         -2.8127e+00,  7.8190e-01,

In [10]:
model = encoderTransformer(1, 1, 15, 'cpu')

def train(model, xtrain, ytrain, xtest, ytest, epochs, batch_size):

    train_loss = []
    val_loss = []

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_function = nn.CrossEntropyLoss()

    for _ in range(epochs):
        tot_loss = 0
        for i in range(0, len(xtrain), batch_size):
            optimizer.zero_grad()
            x = xtrain[i:min(i+batch_size, len(xtrain))]
            y = torch.tensor(ytrain[i:min(i+batch_size, len(ytrain))]).float()
            y_pred = model(x)
            loss = loss_function(y_pred, y)
            tot_loss += loss.item()
            loss.backward()
            optimizer.step()

        print(f"Epoch: {_+1}, Train Loss: {tot_loss/len(xtrain)}")
        train_loss.append(tot_loss/len(xtrain))

        if _ % 10 == 0:
            tot_loss = 0
            for i in range(0, len(xtest), batch_size):
                x = xtest[i:min(i+batch_size, len(xtest))]
                y = torch.tensor(ytest[i:min(i+batch_size, len(ytest))]).float()
                y_pred = model(x)
                loss = loss_function(y_pred, y)
                tot_loss += loss.item()
            print(f"Epoch: {_+1}, Validation Loss: {tot_loss/len(xtest)}")
            print("--------------------------------------------------")
            print("Real label:" + str(y[0]))
            print("Predicted label:" + str(y_pred[0]))
            print("--------------------------------------------------")
            val_loss.append(tot_loss/len(xtest))

    return train_loss, val_loss

train(model, X_train, y_train, X_test, y_test, 20, 100)

1 fc layers
output layer count: 1
